In [37]:
# Logic Building Assistace

# need llm
!pip install langgraph

In [38]:
!pip install langchain

In [39]:
# create state

#START → Understand Problem → Generate Logic → Generate Code → Test Code → END


from typing import TypedDict

class CodeGenState(TypedDict):
  promblem:str
  logic:str
  code:str
  test:str



In [40]:
# we need llm so
!pip install langchain-groq

In [41]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [42]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b" , api_key=os.getenv("GROQ_API_KEY"))

In [43]:
# we we deal with stateGraph

from langgraph.graph import StateGraph

In [44]:
graph = StateGraph(CodeGenState)

In [45]:
def buildlogic(state:CodeGenState) -> CodeGenState:
  # we need to extract promblem
  problem = state['promblem']

  # create prompt
  prompt = f'on the basis of given problem {problem} you need to gnearte the logic to solved this problem possible way and logic and algorithum the main logic to impove proggrma '

  # let genrate logic

  logic = llm.invoke(prompt)

  # now we need to update logic
  state['logic'] = logic.content

  return state


In [46]:
def gencode(state:CodeGenState) -> CodeGenState:
  # we need to extract logic and problem
  logic = state['logic']
  problem = state['promblem']

  # now we have both problem and logic
  prompt = f"we have problem {problem} and logic{logic} you need to genrate code based on problem and logic "

  # let genrate code
  code = llm.invoke(prompt)

  state['code'] = code.content

  return state

In [47]:
# test node
def testnode(state:CodeGenState) -> CodeGenState:
  #we need to extract the code
  code = state['code']
  logic = state['logic']
  problem = state['promblem']

  # need to text code based on logic and problem statement
  prompt = f'we need to test code {code} based on problem{problem} and logic {logic}'


  # need to test this worflow

  test = llm.invoke(prompt)

  state['test'] = test.content

  return test


In [48]:
# lets create node

graph.add_node("buildlogic",buildlogic)

graph.add_node("gencode", gencode)

graph.add_node("testnode",testnode )


In [49]:
# let add edge

from langgraph.graph import  START , END

graph.add_edge(START, 'buildlogic')
graph.add_edge('buildlogic', 'gencode')
graph.add_edge('gencode', 'testnode')
graph.add_edge('testnode', END)


In [50]:
workflow = graph.compile()

In [51]:
workflow.invoke({"promblem":"Find the sum of first N natural numbers"})

{'promblem': 'Find the sum of first N natural numbers',
 'logic': '<think>\nFirst, I need to understand the problem: finding the sum of the first N natural numbers.\n\nI recall that there\'s a straightforward arithmetic approach for this. The sum can be calculated using the formula S = N/2 * (N + 1).\n\nTo implement this in a program, I can start by prompting the user to input the value of N.\n\nNext, I\'ll validate the input to ensure it\'s a positive integer. If the input is invalid, I\'ll display an error message and prompt the user again.\n\nOnce a valid N is provided, I\'ll compute the sum using the formula.\n\nFinally, I\'ll display the computed sum to the user.\n\nThis approach ensures the program is efficient and user-friendly.\n</think>\n\nTo find the sum of the first \\( N \\) natural numbers, we can use a straightforward arithmetic approach. Here\'s a step-by-step explanation of the logic and algorithm:\n\n### **Problem Statement**\nFind the sum of the first \\( N \\) natura